### 1. Load CATPCHA images and get the transformed (X,y) data

In [1]:
import torch

from helper_functions.data_transformation import get_transformed_data_v2, get_transformed_data_for_captcha_evaluation

train_folder = "../data/train/combine"
test_folder = "../data/test/combine"
cached_data_path = "../data/prepared_data_cnn_v2.pth"

try:
    # Load the prepared data from cache if it already exists
    X_train, y_train, X_test, y_test, X_test_captcha, y_test_captcha, failed_segmentation_info = torch.load(cached_data_path)
    print("Loaded prepared data from cache.")

except FileNotFoundError:
    # Else prepare the data and save it
    X_train, y_train = get_transformed_data_v2(train_folder, is_train=True)
    X_test, y_test = get_transformed_data_v2(test_folder, is_train=False)
    X_test_captcha, y_test_captcha, failed_segmentation_info = get_transformed_data_for_captcha_evaluation(test_folder)

    torch.save((X_train, y_train, X_test, y_test, X_test_captcha, y_test_captcha, failed_segmentation_info), cached_data_path)
    print("Saved prepared data to cache.")


C:\Users\ongka\AppData\Local\Temp\ipykernel_17752\4202067423.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  X_train, y_train, X_test, y_test, X_test_captcha, y_test_ca

Loaded prepared data from cache.


### 2. Instantiate basic CNN model and train it

In [2]:
from models.cnn_v2 import CNN, CNNParams
from helper_functions.data_transformation import IMG_HEIGHT, IMG_WIDTH


# Instantiate model
cnn_params: CNNParams = {
    "num_classes": 36,
    "input_shape": (1, IMG_HEIGHT, IMG_WIDTH), # (channels, height, width)
    "learning_rate": 1.1*1e-3,
    "num_epochs": 20,
    "batch_size": 500,
}
cnn = CNN(cnn_params)

# Train model
cached_model_path = "../trained_models/cnn_v2_86_char_accuracy.pth"
try:
    # Load the trained model from cache if it already exists
    cnn.load_trained_model(cached_model_path)
    print("Loaded trained model from cache.")
except FileNotFoundError:
    # Else train the model and save it
    cnn.train_model(X_train, y_train)
    cnn.save_trained_model(cached_model_path)
    print("Saved trained model to cache.")


Loaded trained model from cache.


C:\Users\ongka\OKW_NUS\captcha\playground\models\cnn_v2.py:106: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_state_dict, epoch_losses = torch.load(model_path)


### 3. Evaluate CNN model

In [3]:
import json
from helper_functions.evaluation import evaluate_character_performance, evaluate_captcha_performance

character_performance = evaluate_character_performance(cnn.model, X_test, y_test)
print(f"Character level performance (skipping failed segmentations): {json.dumps(character_performance, indent=2)}\n")

captcha_performance = evaluate_captcha_performance(cnn.model, X_test_captcha, y_test_captcha, failed_segmentation_info)

Character level performance (skipping failed segmentations): {
  "accuracy": 0.8637689614935823,
  "precision": 0.8652564881394587,
  "recall": 0.8618724844604085,
  "f1_score": 0.8626544240256913
}

Captcha level performance: {
  "segmentation_accuracy": 0.9091869060190074,
  "captcha_accuracy (skipping SF)": 0.540650406504065,
  "captcha_accuracy (including SF)": 0.49155227032734955,
  "character_accuracy (including SF)": 0.7833333333333333
}

